# Understanding Feature Hashing Trick
사용자가 검색한 키워드 list와 같은 feature는 매우 sparse, cardinality 높고 가변적이기 때문에,
고정된 feature size를 사용해야하는 machine learning에서 사용하기 어렵다.
Hashing Trick이란 방법은, 이러한 feature를 고정된 size의 vector로 만들어 준다.
본 tutorial은 sample data를 통해 Hashing trick을 수행해보고, 어떻게 동작하는지 이해하고자 한다.

## Problem details
1. 고정된 수의 클래스 변수를 벡터화 하는 방법은 고정된 차원으로 각 변수당 1 또는 0으로 할당한다 (Dummy or One-hot)
    - A, B, C 3개의 클래스를 가진 데이터가 [B, C, B, ...]로 나타났다면, 벡터로 변환하면 다음과 같다
    - [ [0, 1, 0] ,[0, 0, 1], [0, 1, 0], ... ]
2. 문제의 사용자가 검색한 키워드 같은 경우, 고정된 수의 데이터를 가지지 않는다.
    - 주어진 데이터에서 모든 키워드에 index를 할당하고 그 수만큼의 차원으로 위와같이 할당한다면??
        - 업데이트된 데이터에 새로운 키워드가 나타나면, 차원이 계속 늘어나게 됨. (고정된 피처를 만 들 수 없음)
        - embedding 방법을 사용할 수 있지만 새로 나타난 키워드와 빈도가 낮은 키워드에 대한 관리가 필요.
    - 사용자가 몇개의 키워드를 검색할지 모른다.

In [1]:
from sklearn.feature_extraction.text import HashingVectorizer

import pandas as pd
import numpy as np

## Load data
"Most Searched Words on Google" data from https://www.mondovo.com/keywords/

구글에서 가장 많이 검색된 1000개의 키워드 데이터

In [2]:
data = pd.read_table('google_keywords_data.txt', encoding="latin-1", 
                         sep="\t", names=['rank','keyword','volume','cpc'], header=0)

In [3]:
data['volume'] = data['volume'].map(lambda x: int(x.replace(',', '')))
data['cpc'] = data['cpc'].map(lambda x: float(x.replace('$', '')))
print(data.shape)
data.head()

(1000, 4)


,rank,keyword,volume,cpc
0,1,facebook,2147483647,0.02
1,2,youtube,1680000000,0.20
2,3,google,923000000,1.07
3,4,gmail,506000000,0.15
4,5,hotmail,506000000,0.11


## Generate toy data set
500명의 user당 검색 키워드 총 수를 1~50개 추출하고,
1000개의 키워드 중 volume기반 확률 추출로 키워드를 할당해준다.

In [4]:
np.random.seed(1050)
prob = np.log(data.volume)/sum(np.log(data.volume))
user_list = []
for u in range(500):
    search_cnt = np.random.choice(50, 1)[0]
    keywrod_list = list(np.random.choice(data.keyword, search_cnt, p=prob))
    user_list.append(keywrod_list)

** (* 아래 실험 처리는 처음 Hashing Trick에 대한 잘못된 이해로 부터 나옴) ** <br><br>
비교를 위해 index 1, 2, 3 번의 user는 0번 user의 keyword를 각각 sort하고, 마지막 5개를 drop 하고, 처음 5개를 두번 반복한다.

index 0: "D, C, B, A" <br>
index 1: "A, B, C, D" <br>
index 2: "D, C, B"    <br>
index 3: "D, C, D, C" <br>

1. index 0과 1은 string으로 비교하였을 때, 서로 다른 string이다. feature hashing 후 같아 질까??
2. index 2는 0에 포함되는 string을 가진다. feature hashing 후에도 포함될까??
3. index 3과 같이 반복되는 키워드는 어떻게 될까??

In [5]:
user_list[1] = sorted(user_list[0])
user_list[2] = user_list[0][:-5]
user_list[3] = user_list[0][:5] + user_list[0][:5]
for i in range(5): print(user_list[i], "\n")

['hangout', 'chatting', 'cars', 'daily mail', 'uninterrupted power supply', 'mobo market', '9apps', 'whatsapp download', 'verizon', 'fallout 4', 'hillary clinton', 'aol mailbox', 'chatroulette', 'badoo', 'ansaa', 'bernie sanders', 'aricanas', 'cool math game', 'chelsea', 'toys r us'] 

['9apps', 'ansaa', 'aol mailbox', 'aricanas', 'badoo', 'bernie sanders', 'cars', 'chatroulette', 'chatting', 'chelsea', 'cool math game', 'daily mail', 'fallout 4', 'hangout', 'hillary clinton', 'mobo market', 'toys r us', 'uninterrupted power supply', 'verizon', 'whatsapp download'] 

['hangout', 'chatting', 'cars', 'daily mail', 'uninterrupted power supply', 'mobo market', '9apps', 'whatsapp download', 'verizon', 'fallout 4', 'hillary clinton', 'aol mailbox', 'chatroulette', 'badoo', 'ansaa'] 

['hangout', 'chatting', 'cars', 'daily mail', 'uninterrupted power supply', 'hangout', 'chatting', 'cars', 'daily mail', 'uninterrupted power supply'] 

['craigslist', 'traductor', 'phim sex', 'wells fargo', 'ap

list의 keywords를 space separate string으로 변환한다.

In [6]:
user_item = [' '.join(l) for l in user_list]
for i in range(5): print(user_item[i], "\n")

hangout chatting cars daily mail uninterrupted power supply mobo market 9apps whatsapp download verizon fallout 4 hillary clinton aol mailbox chatroulette badoo ansaa bernie sanders aricanas cool math game chelsea toys r us 

9apps ansaa aol mailbox aricanas badoo bernie sanders cars chatroulette chatting chelsea cool math game daily mail fallout 4 hangout hillary clinton mobo market toys r us uninterrupted power supply verizon whatsapp download 

hangout chatting cars daily mail uninterrupted power supply mobo market 9apps whatsapp download verizon fallout 4 hillary clinton aol mailbox chatroulette badoo ansaa 

hangout chatting cars daily mail uninterrupted power supply hangout chatting cars daily mail uninterrupted power supply 

craigslist traductor phim sex wells fargo apps backpages zeta bookmyshow flipkart man united traductor gmail taylor swift livecricket crikbuzz angelina jolie emoji keyboard isis 



## Implementation hashing vectorize

In [7]:
# string to hashing vector
hashing_vec_1 = HashingVectorizer(binary=True, norm=None, n_features=20)
hashing_vec_2 = HashingVectorizer(binary=False, norm=None, n_features=20)
hashing_vec_1
print("see more details of paramter options:", 
'https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html')

see more details of paramter options: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html


In [8]:
# string data를 input하여 vector화 한다.
hashed_text_1 = hashing_vec_1.fit_transform(user_item)
hashed_text_2 = hashing_vec_2.fit_transform(user_item)
## sparse matrix type의 output을 얻는다.
hashed_text_2

<500x20 sparse matrix of type '<class 'numpy.float64'>'
	with 7147 stored elements in Compressed Sparse Row format>

In [9]:
# sparse to dense
## data를 확인하기 위해 dense matrix로 변경한다.
hashed_df_1 = pd.DataFrame(hashed_text_1.todense())
hashed_df_2 = pd.DataFrame(hashed_text_2.todense())

### binary true hashing
1. index 1은 0의 sorted list로 동일한 vector를 return한다.
2. index 2는 0의 마지막 단어 5개를 드랍했는데, colume 3, 7, 15 3개가 다르다. <br>
    - index 0이 index 2를 포함한다.
3. index 3은 0의 첫 5개의 단어를 두번 반복했는데, index 0에 포함된다. <br>
    - 또한, index 3은 index 2에도 포함된다. <br>

즉, 단어에 따라 정보가 보존되는 것으로 보인다.

In [10]:
hashed_df_1[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0


### binary false hasing 
양수와 음수가 공존하는데, 값이 있고 없고는 위 binary true와 동일하다. <br>
index 3의 경우 2배가 되었다.

In [11]:
hashed_df_2[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.0,1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,1.0,2.0,-1.0,1.0,-1.0,0.0,1.0,0.0,1.0,-1.0,-1.0
1,1.0,0.0,1.0,-1.0,0.0,-1.0,0.0,1.0,0.0,1.0,2.0,-1.0,1.0,-1.0,0.0,1.0,0.0,1.0,-1.0,-1.0
2,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,-1.0,-1.0,0.0,1.0,1.0,-1.0,-1.0
3,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,0.0,2.0,0.0,-2.0
4,0.0,1.0,1.0,0.0,1.0,0.0,-3.0,1.0,1.0,-1.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0


## hashing each keyword
user당 "keyword1, keyword2, keyword3, ..." string으로 해싱을 수행하였는데, <br>
각 keyword에 대하여 hashing을 수행하여 보자.

In [12]:
hashed_text_3 = hashing_vec_1.fit_transform(data.keyword.values)
hashed_text_4 = hashing_vec_2.fit_transform(data.keyword.values)
hashed_df_3 = pd.DataFrame(hashed_text_3.todense())
hashed_df_4 = pd.DataFrame(hashed_text_4.todense())

1. 아래 df_3, df_4의 index 2, 3(keyword google, gmail)을 비교해 보면 같은 colume에 값을 가지고 binary false의 경우 하나는 1 하나는 -1이 된다.
    - 즉, test를 위해 차원을 20으로 많이 줄였고, colume이 중복되는 단어에 있어서 하나는 1 하나는 -1이 되는 것으로 보인다.
    - 그렇다면, 3단어 이상이 중복 된다면??
        - 각 colume의 max, min이 각각 2, -2까지 있는 것으로 보아...
        - ~~3번째 중복단어는 2, 4번째 중복단어는 -2, 5번째는 3, ... 으로 되는 것으로 보인다.~~
        - 단어는 무조건 1 or -1로 떨어지고 "bee bee" 같이 중복된 단어가 반복된 경우 값이 2 또는 -2 그 이상이 될 수 있는 것이다.
        - hash 값의 colume index를 할당하는 로직이 mod 컬럼수 이므로 여러 단어가 중복되게 같은 colume에 같은 값으로 할당 될 수 있는 것이다.
2. index 11(keyword 'facebook login')을 비교해 보면, 다른 keyword들은 하나의 colume에 값을 같는데, 11번은 colume 13, 15에 값을 갖는다.
    - 즉, space로 tokenize되어 각 단어가 해당 colume에 위치한 것이다.
    - HashingVectorizer 파라미터중 default "token_pattern=’(?u)\b\w\w+\b’"의 '\b'에 의해 space로 keyword를 분리하는 것으로 보인다.
    - index 1의 facebook keyword에 할당된 colume을 보면 15번이고, index 11의 colume이 13, 15인 것으로 보아 login keyword는 colume 13에 위치하는 것으로 보인다.

In [13]:
data.keyword.values[:13]

array(['facebook', 'youtube', 'google', 'gmail', 'hotmail', 'xxnx',
       'xvideos', 'amazon', 'translator', 'xxx', 'pornos',
       'facebook login', 'yahoo'], dtype=object)

In [14]:
hashed_df_3[:13]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
hashed_df_4[:13]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
hashed_df_4[hashed_df_4.iloc[:,0]==-2]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
654,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
data.keyword.values[654]

'bee bee'

## sum up each keyword vector by user
각 user의 keyword list의 vector를 합산한다.

In [18]:
# each keyword vector to dictionary
## 3: binary true, 4: binary false
hashed_dict_3 = {data.keyword.values[i]: v for i, v in enumerate(hashed_df_3.values)}
hashed_dict_4 = {data.keyword.values[i]: v for i, v in enumerate(hashed_df_4.values)}

In [19]:
print(user_list[0])

['hangout', 'chatting', 'cars', 'daily mail', 'uninterrupted power supply', 'mobo market', '9apps', 'whatsapp download', 'verizon', 'fallout 4', 'hillary clinton', 'aol mailbox', 'chatroulette', 'badoo', 'ansaa', 'bernie sanders', 'aricanas', 'cool math game', 'chelsea', 'toys r us']


In [20]:
sum([hashed_dict_3[i] for i in user_list[0]])

array([3., 0., 2., 1., 2., 1., 0., 1., 0., 1., 2., 3., 3., 1., 3., 1., 2.,
       1., 1., 1.])

In [21]:
sum([hashed_dict_4[i] for i in user_list[0]])

array([ 1.,  0.,  1., -1.,  0., -1.,  0.,  1.,  0.,  1.,  2., -1.,  1.,
       -1.,  0.,  1.,  0.,  1., -1., -1.])

In [22]:
hashed_df_2.iloc[0,:].values

array([ 1.,  0.,  1., -1.,  0., -1.,  0.,  1.,  0.,  1.,  2., -1.,  1.,
       -1.,  0.,  1.,  0.,  1., -1., -1.])

1. hashed_df_2 즉, user의 keyword list string 을 binary false로 hashing vector와 각 단어를 hashing한 후 합산한 결과와 동일 한 것을 확인 할 수 있다.
2. 첫번째(hashed_dict_3)와 같은 결과를 얻기 위해서는 HashingVectorizer를 다음과 같이 만들어야 한다.
    - alternate_sign 값을 False로 해야한다.
    - HashingVectorizer(binary=False, norm=None, alternate_sign=False, n_features=20)

## deep understanding
murmurhash3 hash 함수만 사용하여 단어를 벡터화 해보자

In [23]:
## 아래 두개의 library에서 murmurhash3 hash function을 제공한다.
from sklearn.utils import murmurhash3_32
#import mmh3

In [24]:
## exam keywords
### 각각의 keyword를 hashing vector화 해보자
keylist = ["A0","B1","C2","D3","E4","F5","G6","H7","I8","J9"]

In [25]:
## murmurhash3 함수가 string 값을 받아 어떤 output을 산출하는지 확인해 보자.
### hash가 스트링을 +,- 10자리의 10진수의 표현으로 정수를 산출한다.
hashed_word = [murmurhash3_32(i) for i in keylist]
print(hashed_word)

[-518933037, -1857517943, 129054934, 429647475, -1174188626, 715794145, -1113753596, 1522157951, -1579529551, 176510599]


In [26]:
## 10개의 keyword를 5차원의 피처로 만들어 보자.
### 먼저 각 keyword에 mod값을 사용하여 index를 할당한다.
n_features = 5
hashed_feat = [abs(i) % 5 for i in hashed_word]
print(hashed_feat)

[2, 3, 4, 0, 1, 0, 1, 1, 1, 4]


In [27]:
## binary true 일경우
feat= np.zeros([10,5])
for i, f in enumerate(hashed_feat): 
    feat[i, f] = 1
feat

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [28]:
## binary false 일경우
### hash 값의 sign 값 즉, 음수이면 -1, 양수이면 +1을 할당한다.
for i, f in enumerate(hashed_feat): 
    sign = (hashed_word[i] >= 0) * 2 - 1
    feat[i, f] = 1 * sign
feat

array([[ 0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

HashingVectorizer 함수를 사용해서 비교해보면, 위에서 직접 vector화한 결과와 상이한 것을 볼 수 있다. <br>
위 함수의 원문 소스를 확인해보니, sklearn의 murmurhash3_32를 동일하게 사용하지만 차이점으로 cimport로 사용한다. <br> cimport 함수의 값을 직접 확인해보는 방법을 몰라 확실하지 않지만, hash 값에서 차이가 있는 것으로 보이고, mod와 sign 값으로 +,-를 할당하는 것은 동일하다.

In [29]:
hv = HashingVectorizer(binary=False, norm=None, n_features=5)
tmp = hv.fit_transform(keylist)
pd.DataFrame(tmp.todense())

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,-1.0
2,-1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-1.0,0.0
4,0.0,-1.0,0.0,0.0,0.0
5,0.0,-1.0,0.0,0.0,0.0
6,-1.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,-1.0
9,0.0,0.0,0.0,-1.0,0.0
